# Import



In [ ]:
# general

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import pickle


# local

import evaluation, preprocessor, univariate, tuning

from evaluation import (    
    make_random_list,
    time_points,
    
    plot_c_index_ci,
    plot_td_auc_no_ci,
    plot_td_bs_no_ci,
    
    get_best_cutoff,
    plot_km_together,
    plot_km_original,
    
    plot_predictions,
    plot_personalized_predictions,

    plot_beeswarm_per_features,
    get_ylabels
)

from preprocessor import (
    ENCODER,
    onehot_encoder,
    ordinal_encoder,
    preprocessor_test
)

from univariate import (
    plot_cumulative_dynamic_auc_together,
    plot_cumulative_dynamic_auc,
    get_univariate_table
)

# sksurv metrics

from sksurv.metrics import (
    concordance_index_censored, 
    cumulative_dynamic_auc, 
    integrated_brier_score
)
from sksurv.metrics import (
    as_concordance_index_ipcw_scorer, # Uno's C-index
    as_cumulative_dynamic_auc_scorer, # Harrell's C-index
    as_integrated_brier_score_scorer,
)

# sksurv MODELS

from sksurv.linear_model import (
    CoxPHSurvivalAnalysis, 
    CoxnetSurvivalAnalysis,
)
from sksurv.ensemble import (
    RandomSurvivalForest, 
    ComponentwiseGradientBoostingSurvivalAnalysis, 
    GradientBoostingSurvivalAnalysis
)
from sksurv.tree import (
    SurvivalTree
)
from sksurv.svm import (
    FastSurvivalSVM, 
    FastKernelSurvivalSVM
)
from sksurv.kernels import clinical_kernel, ClinicalKernelTransform

# sksurv OTHERS

from sksurv.nonparametric import kaplan_meier_estimator
from sksurv.util import Surv

# SKLEARN

from sklearn import set_config
set_config(display="text")  # displays text representation of estimators
from sklearn.exceptions import FitFailedWarning

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.linear_model import BayesianRidge

from sklearn.preprocessing import PowerTransformer

from sklearn.model_selection import GridSearchCV, KFold, ShuffleSplit

from sklearn.inspection import permutation_importance

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

# Explain

import shap

# Color changer

In [ ]:
# tab20

tab20_colors = plt.cm.tab20.colors
tab20_cycle = plt.cycler(color=tab20_colors)
plt.rcParams['axes.prop_cycle'] = tab20_cycle

In [ ]:
# tab10

tab10_colors = plt.cm.tab10.colors
tab10_cycle = plt.cycler(color=tab10_colors)
plt.rcParams['axes.prop_cycle'] = tab10_cycle

# Prepare

In [ ]:
# load

data = pd.read_table('data/data.csv', sep=',', header=0)

In [ ]:
# split

data_train = data[data['id'] == 'SEER']
data_test = data[data['id'] == 'External']

In [ ]:
# feature name lists

feature_names = ['Age','T category','N category','M category','AJCC stage','Extent','Grade','Tumor size','Surgery','Hysterectomy','Chemotherapy','Radiotherapy']

In [ ]:
# Train targets

X_train = data_train[feature_names]

y_OS_train = Surv.from_arrays(data_train['ACD'], data_train['Time'])
y_DSS_train = Surv.from_arrays(data_train['DSD'], data_train['Time'])

# avoid zeros

y_OS_train[13] = (True, 1e-10) # fake time 0 -> 1e-10
y_DSS_train[13] = (True, 1e-10) # fake time 0 -> 1e-10


In [ ]:
# Test targets

X_test = data_test[feature_names]

y_OS_test = Surv.from_arrays(data_test['ACD'], data_test['Time'])
y_DSS_test = Surv.from_arrays(data_test['DSD'], data_test['Time'])

# Preprocessing for train set

## Encoding

In [ ]:
X_train_encode = ENCODER(X_train)
display(X_train_encode)

## Imputing

In [ ]:
imputer = IterativeImputer(estimator=BayesianRidge(),
                           max_iter=1000,
                           initial_strategy='mean',
                           sample_posterior=True,
                           random_state=2024) #

X_train_impute = imputer.fit_transform(X_train_encode)
X_train_impute = pd.DataFrame(X_train_impute, columns = X_train_encode.columns)
display(X_train_impute)

## Scaling

In [ ]:
scaler = PowerTransformer()

X_train_scale = scaler.fit_transform(X_train_impute)
X_train_scale = pd.DataFrame(X_train_scale, columns=X_train_impute.columns)
display(X_train_scale)

# Preprocessing for test set

In [ ]:
X_test_scale = preprocessor_test(X_test, ENCODER, imputer, scaler)

# Final datasets

In [ ]:
X_train_final = X_train_scale
X_test_final = X_test_scale

# KM

## SEER

In [ ]:
seer_dss = plot_km_original(y_DSS_train, 'DSS')
seer_os = plot_km_original(y_OS_train, 'OS')
plt.legend(loc='lower left')

plt.savefig('KM SEER.pdf', format='pdf', bbox_inches = 'tight')
plt.show()

## External

In [ ]:
exte_dss = plot_km_original(y_DSS_test, 'DSS')
exte_os = plot_km_original(y_OS_test, 'OS')
plt.legend(loc='lower left')

plt.savefig('KM External.pdf', format='pdf', bbox_inches = 'tight')
plt.show()

# Univariate FS using baseline models

In [ ]:
cv_ = ShuffleSplit(n_splits = 100, test_size = 0.5, random_state = 1437)

In [ ]:
estimator_list = [
    CoxPHSurvivalAnalysis(), 
    CoxnetSurvivalAnalysis().set_params(**{'fit_baseline_model': True}), 
    RandomSurvivalForest().set_params(**{'random_state': 1437, 'n_estimators': 50, 'max_depth': 500}), 
    GradientBoostingSurvivalAnalysis().set_params(**{'random_state': 1437, 'learning_rate': 0.5, 'n_estimators': 150}),
    SurvivalTree().set_params(**{'random_state': 1437, 'max_depth': 100})
]

name_list = ['CoxPH', 'CoxNet', 'RSF', 'GBM', 'ST']

table, annot_table = get_univariate_table(estimator_list, 
                                          X_train_final, 
                                          name_list, 
                                          y_OS_train, 
                                          cv = cv_)

In [ ]:
ax = sns.heatmap(data = table.transpose(),
                 annot = annot_table.transpose(),
                 fmt = '', 
                 cmap= 'coolwarm',
                 center=0.5)
ax.set_xlabel('Model\n', fontsize = 14)
ax.set_ylabel('Feature', fontsize = 14)
ax.set_title('Univariate C-indices\n', fontsize = 16, fontweight='bold')
ax.xaxis.tick_top()
ax.xaxis.set_label_position('top')

fig = ax.get_figure()
fig.set_size_inches(10,10)
fig.show()
fig.savefig('Univariate FS.pdf', format='pdf', bbox_inches = 'tight')

# Train & evaluate FS models

In [ ]:
X_train_coxph = X_train_final[['Age', 'N category', 'Hysterectomy', 'Extent', 'AJCC stage', 'M category', 'Chemotherapy', 'Grade', 'Radiotherapy_EBRT', 'Surgery_PR', 'Radiotherapy_No/Unknown', 'T category']]
X_train_coxnet = X_train_final[['Age', 'N category', 'Hysterectomy', 'Extent', 'AJCC stage', 'Chemotherapy', 'M category', 'Radiotherapy_EBRT', 'Grade', 'Surgery_PR', 'Radiotherapy_No/Unknown', 'T category']]
X_train_rsf = X_train_final[['Age', 'Extent', 'N category', 'Hysterectomy', 'Surgery_PR', 'Chemotherapy', 'M category', 'Radiotherapy_RAI', 'Surgery_USO', 'Tumor size', 'Radiotherapy_EBRT', 'Grade', 'AJCC stage']]
X_train_gbm = X_train_final[['Age', 'N category', 'M category', 'Extent', 'Grade', 'Tumor size', 'Hysterectomy', 'Chemotherapy', 'Surgery_PR', 'Radiotherapy_EBRT', 'Radiotherapy_No/Unknown']]
X_train_sdt = X_train_final[['Extent', 'Age', 'Hysterectomy', 'N category', 'Surgery_PR', 'Chemotherapy', 'Tumor size', 'M category', 'Radiotherapy_RAI', 'Grade', 'Surgery_USO', 'Radiotherapy_EBRT']]

X_test_coxph = X_test_final[['Age', 'N category', 'Hysterectomy', 'Extent', 'AJCC stage', 'M category', 'Chemotherapy', 'Grade', 'Radiotherapy_EBRT', 'Surgery_PR', 'Radiotherapy_No/Unknown', 'T category']]
X_test_coxnet = X_test_final[['Age', 'N category', 'Hysterectomy', 'Extent', 'AJCC stage', 'Chemotherapy', 'M category', 'Radiotherapy_EBRT', 'Grade', 'Surgery_PR', 'Radiotherapy_No/Unknown', 'T category']]
X_test_rsf = X_test_final[['Age', 'Extent', 'N category', 'Hysterectomy', 'Surgery_PR', 'Chemotherapy', 'M category', 'Radiotherapy_RAI', 'Surgery_USO', 'Tumor size', 'Radiotherapy_EBRT', 'Grade', 'AJCC stage']]
X_test_gbm = X_test_final[['Age', 'N category', 'M category', 'Extent', 'Grade', 'Tumor size', 'Hysterectomy', 'Chemotherapy', 'Surgery_PR', 'Radiotherapy_EBRT', 'Radiotherapy_No/Unknown']]
X_test_sdt = X_test_final[['Extent', 'Age', 'Hysterectomy', 'N category', 'Surgery_PR', 'Chemotherapy', 'Tumor size', 'M category', 'Radiotherapy_RAI', 'Grade', 'Surgery_USO', 'Radiotherapy_EBRT']]

## train

In [ ]:
coxph = CoxPHSurvivalAnalysis()
coxnet = CoxnetSurvivalAnalysis().set_params(**{'l1_ratio': 0.005, 'fit_baseline_model': True})
rsf = RandomSurvivalForest().set_params(**{'random_state': 56, 'max_depth': 680, 'n_estimators': 4})
gbm = GradientBoostingSurvivalAnalysis().set_params(**{'random_state': 23, 'dropout_rate': 0.1, 'learning_rate': 0.1, 'max_depth': 1450, 'n_estimators': 70, 'subsample': 0.5})
sdt = SurvivalTree().set_params(**{'random_state': 53, 'splitter': 'random', 'max_depth': 40, 'min_samples_leaf': 6, 'min_samples_split': 20})

coxph.fit(X_train_coxph, y_OS_train)
coxnet.fit(X_train_coxnet, y_OS_train)
rsf.fit(X_train_rsf, y_OS_train)
gbm.fit(X_train_gbm, y_OS_train)
sdt.fit(X_train_sdt, y_OS_train)

## get lists

In [ ]:
estimator_list = [coxph, coxnet, rsf, gbm, sdt]
X_test_list = [X_test_coxph, X_test_coxnet, X_test_rsf, X_test_gbm, X_test_sdt]
X_train_list = [X_train_coxph, X_train_coxnet, X_train_rsf, X_train_gbm, X_train_sdt]
name_list = ['CoxPH', 'CoxNet', 'RSF', 'GBM', 'ST']

## CI w/ 95% CI

In [ ]:
random_list = make_random_list(n_samples = 100000,  # make sure unique
                               seed = 1437)

### Testing set

In [ ]:
ax1 = plot_c_index_ci(estimator_list, 
                      X_test_list, 
                      y_OS_test, 
                      name_list, 
                      n_samples = 1000, 
                      random_list = random_list)

In [ ]:
ax = ax1
ax.set_title('Model performance in testing set', fontsize = 12)
fig = ax.get_figure()
fig.set_size_inches(6,4)
fig.savefig('CI test.pdf', format='pdf', bbox_inches = 'tight')

### Training set

In [ ]:
ax2 = plot_c_index_ci(estimator_list, 
                      X_train_list, 
                      y_OS_train, 
                      name_list, 
                      n_samples = 1000, 
                      random_list = random_list)

In [ ]:
ax = ax2
ax.set_title('Model performance in training set', fontsize = 12)
fig = ax.get_figure()
fig.set_size_inches(6,4)
fig.savefig('CI train.pdf', format='pdf', bbox_inches = 'tight')

## mAUC

### Testing set

In [ ]:
ax3 = plot_td_auc_no_ci(estimator_list, 
                        X_test_list, 
                        name_list,
                        y_OS_test, y_OS_train,
                        times = time_points(y_OS_test, 5, 95, 6), # 90% of the times
                        mean = False,
                        xticks = False)

In [ ]:
ax = ax3
ax.set_title('Model performance in testing set', fontsize = 12)
fig = ax.get_figure()
fig.set_size_inches(6,4)
fig.savefig('AUC test.pdf', format='pdf', bbox_inches = 'tight')

### Training set

In [ ]:
ax4 = plot_td_auc_no_ci(estimator_list, 
                        X_train_list, 
                        name_list,
                        y_OS_train, y_OS_train,
                        times = time_points(y_OS_train, 5, 95, 6), # 90% of the times
                        mean = False,
                        xticks = False)

In [ ]:
ax = ax4
ax.set_title('Model performance in training set', fontsize = 12)
fig = ax.get_figure()
fig.set_size_inches(6,4)
fig.savefig('AUC train.pdf', format='pdf', bbox_inches = 'tight')

## IBS

### Testing set

In [ ]:
ax5 = plot_td_bs_no_ci(estimator_list, 
                       X_test_list, 
                       name_list,
                       y_OS_test, y_OS_train,
                       times = time_points(y_OS_test, 5, 95, 6), # 90% of the times
                       ibs = False,
                       xticks = False)

In [ ]:
ax = ax5
ax.set_title('Model performance in testing set', fontsize = 12)
fig = ax.get_figure()
fig.set_size_inches(6,4)
fig.savefig('BS test.pdf', format='pdf', bbox_inches = 'tight')

### Training set

In [ ]:
ax6 = plot_td_bs_no_ci(estimator_list, 
                       X_train_list, 
                       name_list,
                       y_OS_train, y_OS_train,
                       times = time_points(y_OS_train, 5, 95, 6), # 90% of the times
                       ibs = False,
                       xticks = False)

In [ ]:
ax = ax6
ax.set_title('Model performance in training set', fontsize = 12)
fig = ax.get_figure()
fig.set_size_inches(6,4)
fig.savefig('BS train.pdf', format='pdf', bbox_inches = 'tight')

## Risk stratification

### Testing

In [ ]:
for estimator, X_test in zip(estimator_list, X_test_list):
    print(f'{estimator.__class__.__name__}')
    get_best_cutoff(estimator, X_test, y_OS_test) # a value in [25, 75] percentiles

In [ ]:
percentiles = [65,66,75,70,75]
plot_km_together(estimator_list, X_test_list, y_OS_test, name_list, percentiles)

### Training

In [ ]:
for estimator, X_train in zip(estimator_list, X_train_list):
    print(f'{estimator.__class__.__name__}')
    get_best_cutoff(estimator, X_train, y_OS_train) # a value in [25, 75] percentiles

In [ ]:
percentiles = [41,47,72,75,75]
plot_km_together(estimator_list, X_train_list, y_OS_train, name_list, percentiles)

## SHAP

no to much difference for test or train

In [ ]:
# test

explainer = shap.PermutationExplainer(rsf.predict, X_test_rsf)
explanation = explainer(X_test_rsf)

In [ ]:
# train

explainer_ = shap.PermutationExplainer(rsf.predict, X_train_rsf)
explanation_ = explainer(X_train_rsf)

### beeswarm

In [ ]:
plt.clf()
shap.plots.beeswarm(explanation, plot_size = (12, 6), max_display=18, show = True)
shap.plots.beeswarm(explanation, plot_size = (12, 6), max_display=18, show = False)
plt.savefig('beeswarm.pdf', format='pdf', bbox_inches = 'tight')

In [ ]:
plt.clf()
shap.plots.beeswarm(explanation_, plot_size = (12, 6), max_display=18, show = True)
shap.plots.beeswarm(explanation, plot_size = (12, 6), max_display=18, show = False)
plt.savefig('beeswarm_train.pdf', format='pdf', bbox_inches = 'tight')

#### split beeswarm

In [ ]:
for feature_name in ['Age', 'Extent', 'N category', 'Hysterectomy', 'Surgery_PR', 'Chemotherapy', 'M category', 
                     'Radiotherapy_RAI', 'Surgery_USO', 'Tumor size', 'Radiotherapy_EBRT', 'Grade', 'AJCC stage']:
    plot_beeswarm_per_features(explanation, name = feature_name)

### bar

In [ ]:
shap.plots.bar(explanation, clustering=False, max_display=18)

### violin

In [ ]:
shap.plots.violin(explanation, 
                  plot_size = (12, 6), 
                  show = False)
plt.savefig('violin.pdf', format='pdf', bbox_inches = 'tight')

### waterfall

#### High-risk

In [ ]:
explanation_patient=explanation[3]
X_patient=X_test.iloc[3:4]

sorted_ylabels, _ = get_ylabels(explanation_patient, X_patient)

plt.clf()
fig = shap.plots.waterfall(explanation_patient, max_display=18, show = False)
ax_ = fig.get_axes()[0]
tick_labels = ax_.yaxis.get_majorticklabels()
for i in range(len(sorted_ylabels)):
    tick_labels[i].set_color("black")
ax_.set_yticks(np.arange(len(sorted_ylabels)))
ax_.set_yticklabels(sorted_ylabels)

plot = ax_.get_figure()

plot.savefig('No. 3 high-risk shap.pdf', format='pdf', bbox_inches = 'tight')
plt.show()

#### Low-risk

In [ ]:
explanation_patient=explanation[6]
X_patient=X_test.iloc[6:7]

sorted_ylabels, _ = get_ylabels(explanation_patient, X_patient)

plt.clf()
fig = shap.plots.waterfall(explanation_patient, max_display=18, show = False)
ax_ = fig.get_axes()[0]
tick_labels = ax_.yaxis.get_majorticklabels()
for i in range(len(sorted_ylabels)):
    tick_labels[i].set_color("black")
ax_.set_yticks(np.arange(len(sorted_ylabels)))
ax_.set_yticklabels(sorted_ylabels)
plot = ax_.get_figure()

plot.savefig('No. 6 low-risk shap.pdf', format='pdf', bbox_inches = 'tight')
plt.show()

## Predictions

In [ ]:
estimator = rsf
X = X_test_rsf
times = np.arange(0, 360)
best_cop = 5.827909050252443

plot_predictions(estimator, X, times, best_cop, show = False)
plt.savefig('predictions.pdf', format='pdf', bbox_inches = 'tight')

## Personalized prediction

In [ ]:
rs = rsf.predict(X_test_rsf)
best_cut_off_point = np.percentile(rs, 75)
best_cut_off_point

### High-risk

In [ ]:
plt.clf()
estimator = rsf
X = X_test_rsf.iloc[3:4]
times = np.arange(0, 360)
best_cop = 5.827909050252443
plot_personalized_predictions(estimator, X, times, best_cop, show = True)
plot_personalized_predictions(estimator, X, times, best_cop, show = False)
current_axis = plt.gca()
plt.savefig('No. 3 high-risk.pdf', format='pdf', bbox_inches = 'tight')

### Low-risk

In [ ]:
plt.clf()
estimator = rsf
X = X_test_rsf.iloc[6:7]
times = np.arange(0, 360)
best_cop = 5.827909050252443

plot_personalized_predictions(estimator, X, times, best_cop, show = True)
plot_personalized_predictions(estimator, X, times, best_cop, show = False)
current_axis = plt.gca()
plt.savefig('No. 6 low-risk.pdf', format='pdf', bbox_inches = 'tight')

# Deployment

In [ ]:
with open('model.pkl', 'wb') as f:
    pickle.dump(rsf, f)
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
with open('X_test.pkl', 'wb') as f:
    pickle.dump(X_test_rsf, f)